In [16]:
"""
README: Side-by-Side Visualizer for YOLO Detection vs Ground Truth

This script compares YOLO detection results to ground-truth labels per class,
filters for the best-matching examples (by IoU), and creates side-by-side visualizations
of the original, GT-boxed, and detected-boxed images for each class.

How to use:
- Set your YOLO detection label directory, ground truth label directory, and images directory below.
- The script processes each class in parallel and saves the top-10 best matches (by IoU) as side-by-side visual comparisons.
- All output images are stored in per-class subfolders under the specified output directory.

Requirements:
- Python 3.7+
- opencv-python
- concurrent.futures (standard)
- pathlib (standard)
- numpy

Author: Bahadir Akin Akgul
Date: 13.07.2025
"""

import cv2
from pathlib import Path
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

# === SETTINGS ===
labels_dir = Path('PATH_TO_YOLO_DETECTION_LABELS')      # e.g., 'runs/detect/yolov10-txtresults/labels'
gt_dir = Path('PATH_TO_GROUND_TRUTH_LABELS')            # e.g., '.../test/labels'
images_dir = Path('PATH_TO_IMAGES')                     # e.g., '.../test/images'
output_dir = Path('sidebyside-best-matches-filtered')
output_dir.mkdir(parents=True, exist_ok=True)

iou_threshold = 0.5
class_names = ['pedestrian', 'road', 'vehicle']
class_ids = {'pedestrian': 0, 'road': 1, 'vehicle': 2}
colors = {
    0: (0, 255, 255),  # pedestrian - yellow
    1: (0, 0, 0),      # road - black
    2: (0, 0, 255)     # vehicle - red
}

def now(): 
    return datetime.now().strftime('%H:%M:%S')

def calculate_iou(boxA, boxB):
    xA, yA, wA, hA = boxA
    xa1, ya1, xa2, ya2 = xA - wA / 2, yA - hA / 2, xA + wA / 2, yA + hA / 2
    xB, yB, wB, hB = boxB
    xb1, yb1, xb2, yb2 = xB - wB / 2, yB - hB / 2, xB + wB / 2, yB + hB / 2

    inter_x1 = max(xa1, xb1)
    inter_y1 = max(ya1, yb1)
    inter_x2 = min(xa2, xb2)
    inter_y2 = min(ya2, yb2)
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    area_a = (xa2 - xa1) * (ya2 - ya1)
    area_b = (xb2 - xb1) * (yb2 - yb1)
    return inter_area / (area_a + area_b - inter_area + 1e-6)

def draw_boxes(image, boxes, classes, cls_id):
    h, w = image.shape[:2]
    for cls, box in zip(classes, boxes):
        if cls != cls_id:
            continue
        x_c, y_c, bw, bh = box
        x1 = int((x_c - bw / 2) * w)
        y1 = int((y_c - bh / 2) * h)
        x2 = int((x_c + bw / 2) * w)
        y2 = int((y_c + bh / 2) * h)
        cv2.rectangle(image, (x1, y1), (x2, y2), colors[cls], 1)
    return image

def process_file_for_class(cls_name):
    cls_id = class_ids[cls_name]
    class_output = output_dir / cls_name
    class_output.mkdir(parents=True, exist_ok=True)

    results = []
    txt_files = list(labels_dir.glob("*.txt"))

    for txt_file in txt_files:
        stem = txt_file.stem
        image_files = list(images_dir.glob(f"{stem}.*"))
        if not image_files:
            continue
        image_path = image_files[0]
        img = cv2.imread(str(image_path))
        if img is None:
            continue

        gt_path = gt_dir / txt_file.name
        if not gt_path.exists():
            continue

        with open(gt_path) as f:
            gt_lines = [line.strip().split() for line in f if line.strip()]
        gt_classes = [int(l[0]) for l in gt_lines]
        gt_boxes = [list(map(float, l[1:5])) for l in gt_lines]

        with open(txt_file) as f:
            det_lines = [line.strip().split() for line in f if line.strip()]
        det_classes = [int(l[0]) for l in det_lines]
        det_boxes = [list(map(float, l[1:5])) for l in det_lines]

        filtered_det_boxes = []
        filtered_det_classes = []
        matched_gt = set()

        for i, gt_box in enumerate(gt_boxes):
            if gt_classes[i] != cls_id:
                continue
            for j, det_box in enumerate(det_boxes):
                if det_classes[j] != cls_id:
                    continue
                if calculate_iou(gt_box, det_box) >= iou_threshold:
                    filtered_det_boxes.append(det_box)
                    filtered_det_classes.append(cls_id)
                    matched_gt.add(i)
                    break

        match_count = len(matched_gt)
        results.append({
            "match": match_count,
            "image": img,
            "gt_boxes": gt_boxes,
            "gt_classes": gt_classes,
            "det_boxes": filtered_det_boxes,
            "det_classes": filtered_det_classes,
            "name": stem
        })

    top10 = sorted(results, key=lambda x: x["match"], reverse=True)[:10]

    for i, result in enumerate(top10, 1):
        original = result["image"].copy()
        gt_img = draw_boxes(original.copy(), result["gt_boxes"], result["gt_classes"], cls_id)
        det_img = draw_boxes(original.copy(), result["det_boxes"], result["det_classes"], cls_id)
        combined = cv2.hconcat([original, gt_img, det_img])
        save_path = class_output / f"{i:02d}_{result['name']}.jpg"
        cv2.imwrite(str(save_path), combined)
        print(f"[{now()}] ✅ {cls_name.upper()} - Saved: {save_path.name}")

# Parallel processing for each class
with ThreadPoolExecutor(max_workers=8) as executor:
    executor.map(process_file_for_class, class_names)
